### This notebook does not contain all the models, as they were changed in place during the time

In [ ]:
# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# general
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# modelling
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import metrics
import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

# Set plot preference
plt.style.use(style='ggplot')
plt.rcParams['figure.figsize'] = (10, 6)

from sklearn import svm
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from IPython.display import SVG
from statsmodels.tsa.seasonal import seasonal_decompose

from keras import models, layers, optimizers, regularizers

In [ ]:
airbnb = pd.read_csv("airbnb4.csv")

In [ ]:
to_drop = ["Unnamed: 0", "Unnamed: 0.1","longitude", "latitude","id", "air_nodes_true", "coordinates"]

airbnb = airbnb.drop(to_drop, axis = 1)

In [ ]:
numerical_columns = ['accommodates', 'price', 'days_being_host', 'maximum_nights', 'minimum_nights',
                     "availability_365", 'number_of_reviews', "calculated_host_listings_count",
                     "cleaning_fee", "security_deposit", "bathrooms", "extra_people"]

In [ ]:
# Log transforming columns (numerical_raw)
numerical_columns = [i for i in numerical_columns if i not in ["days_being_host", 'availability_365']] # Removing items not to be transformed

for col in numerical_columns:
    airbnb[col] = airbnb[col].astype('float64').replace(0.0, 0.01) # Replacing 0s with 0.01
    airbnb[col] = np.log(airbnb[col])

In [ ]:
to_drop_walk = ["geometry_distance_to_park", "geometry_distance_to_supermarket", "geometry_distance_to_restaurant",
                "geometry_distance_to_station"]

airbnb_walk = airbnb.drop(to_drop_walk, axis = 1)     

In [ ]:
to_drop_walk = ["geometry_distance_to_park", "geometry_distance_to_supermarket", "geometry_distance_to_restaurant"]
airbnb_walk = airbnb.drop(to_drop_walk, axis = 1) 

# Modeling

In [ ]:
# initiate the dataframe, where we will keep the results (up to NN)
results = pd.DataFrame() 
columns = ["Algorithm","TrainMSE", "MSE","TrainR2", "R2", "Parameters"]

results = pd.DataFrame(columns = columns)
i = 0

# Linear Regression

In [ ]:
# Separating X and y
X = airbnb_walk.drop('price', axis=1)
y = airbnb_walk.price

# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

lin_reg = LinearRegression()  
lin_reg.fit(X_train, y_train) # training the algorithm

# Predict
train_pred_lin_reg = lin_reg.predict(X_train)
pred_lin_reg = lin_reg.predict(X_test)

# Check the predictions against the actual values by using the MSE and R-2 metrics:
print("Validation MSE:", round(mean_squared_error(y_test, pred_lin_reg),4))
print("Validation r2:", round(r2_score(y_test, pred_lin_reg),4))

In [ ]:
results.loc[i] = ["Linear Regression",
                  mean_squared_error(y_train, lin_reg.predict(X_train)),
                  mean_squared_error(y_test, pred_lin_reg),
                  r2_score(y_train, lin_reg.predict(X_train)),
                  r2_score(y_test, pred_lin_reg),
                  "-"]

i += 1

# XGBoost

In [ ]:
# Separating X and y
X = airbnb_walk.drop('price', axis=1)
y = airbnb_walk.price

# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train, y_train)

train_pred_xgb = xgb_reg.predict(X_train)
val_pred_xgb = xgb_reg.predict(X_test)

print("\nTraining MSE:", round(mean_squared_error(y_train, train_pred_xgb),4))
print("Validation MSE:", round(mean_squared_error(y_test, val_pred_xgb),4))

print("\nTraining r2:", round(r2_score(y_train, train_pred_xgb),4))
print("Validation r2:", round(r2_score(y_test, val_pred_xgb),4)) # Overfitting

In [ ]:
results.loc[i] = ["Xgboost1",
                  mean_squared_error(y_train, train_pred_xgb),
                  mean_squared_error(y_test, val_pred_xgb),
                  r2_score(y_train, train_pred_xgb),
                  r2_score(y_test, val_pred_xgb),
                  xgb_reg.get_params()]

i += 1

In [ ]:
# Hyperparameter tuning

params = {
    # Parameters that we are going to tune.
    "max_depth" : [8,9,10],
    "min_child_weight": [4,5,6,7],
    'eta':[0.005, .05, .01, 0.1],
    'subsample': [i/10. for i in range(7,11)],
    'colsample_bytree': [i/10. for i in range(7,11)],
    # Other parameters
    "n_estimators" : [100,120, 150] }

scoring = {'R2': 'r2', 'MSE': "neg_mean_squared_error"}
xgb_model = xgb.XGBRegressor()

CV_xgb = GridSearchCV(estimator=xgb_model, param_grid=params,scoring= scoring,refit = "R2", return_train_score=True, cv = 3, n_jobs = -1)
CV_xgb.fit(X_train, y_train)

In [ ]:
xg_reg = CV_xgb.best_estimator_

val_pred_xgb = xg_reg.predict(X_test)

print("Training MSE:", round(mean_squared_error(y_train, xg_reg.predict(X_train)),4))
print("Validation MSE:", round(mean_squared_error(y_test, val_pred_xgb),4))

print("Training r2:", round(r2_score(y_train,  xg_reg.predict(X_train)),4))
print("Validation r2:", round(r2_score(y_test, val_pred_xgb),4))

In [ ]:
results.loc[i] = ["Xgboost2",
                  mean_squared_error(y_train, xg_reg.predict(X_train)),
                  mean_squared_error(y_test, val_pred_xgb),
                  r2_score(y_train,  xg_reg.predict(X_train)),
                  r2_score(y_test, val_pred_xgb),
                  xgb_reg.get_params()]

i += 1

In [ ]:
ft_weights_xgb_reg = pd.DataFrame(xg_reg.feature_importances_, columns=['weight'], index=X_train.columns)
ft_weights_xgb_reg.sort_values('weight', inplace=True)
ft_weights_xgb_reg

In [ ]:
# Plotting feature importances

plt.figure(figsize=(20,20))
plt.barh(ft_weights_xgb_reg.index, ft_weights_xgb_reg.weight, align='center') 
plt.title("Feature importances in the XGBoost model", fontsize=14)
plt.xlabel("Feature importance")
plt.margins(y=0.01)
plt.savefig("Feature Importance XGBoost")
plt.show()

# Random Forest Regression

In [ ]:
# Separating X and y
X = airbnb_walk.drop('price', axis=1)
y = airbnb_walk.price

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

# Scale 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rf = RandomForestRegressor()

rf.fit(X_train, y_train)

train_pred_rf = rf.predict(X_train)
val_pred_rf = rf.predict(X_test)

print("Validation MSE:", round(mean_squared_error(y_test, val_pred_rf),4))
print("Validation r2:", round(r2_score(y_test, val_pred_rf),4))

In [ ]:
results.loc[i] = ["RFR1",
                  mean_squared_error(y_train, train_pred_rf),
                  mean_squared_error(y_test, val_pred_rf),
                  r2_score(y_train, train_pred_rf),
                  r2_score(y_test, val_pred_rf),
                  rf.get_params()]

i += 1

In [ ]:
# Hyperparameter tuning

scoring = {'R2': 'r2', 'MSE': "neg_mean_squared_error"}

param_grid = { 
    'n_estimators': [100,120,150,200],
    'max_features': [10, 20, 30],
    'max_depth' : [8,9,10],
    "min_samples_leaf" :[3,4],
    "min_samples_split":[8,9]
}

CV_rfc_tune = GridSearchCV(estimator=rf, param_grid=param_grid, cv = 3, scoring= scoring,refit = "R2", return_train_score=True, n_jobs =-1)
CV_rfc_tune.fit(X_train, y_train)

In [ ]:
rf_best = CV_rfc_tune.best_estimator_
val_pred_rf = rf_best.predict(X_test)

print("Validation MSE:", round(mean_squared_error(y_test, val_pred_rf),4))
print("Validation r2:", round(r2_score(y_test, val_pred_rf),4))

In [ ]:
results.loc[i] = ["RFR2",
                  mean_squared_error(y_train, rf_best.predict(X_train)),
                  mean_squared_error(y_test, val_pred_rf),
                  r2_score(y_train, rf_best.predict(X_train)),
                  r2_score(y_test, val_pred_rf),
                  rf_best.get_params()]

i += 1

# SVR

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

# Scale 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

svr = SVR(kernel = "rbf", C = 1, gamma = 0.005, epsilon = 0.05)
svr.fit(X_train, y_train)

# Predict
train_pred_svr = svr.predict(X_train)
val_pred_svr = svr.predict(X_test)

print("\nTraining MSE:", round(mean_squared_error(y_train, train_pred_svr),4))
print("Validation MSE:", round(mean_squared_error(y_test, val_pred_svr),4))

print("\nTraining r2:", round(r2_score(y_train, train_pred_svr),4))
print("Validation r2:", round(r2_score(y_test, val_pred_svr),4))

In [ ]:
results.loc[i] = ["SVR1",
                  mean_squared_error(y_train, train_pred_svr),
                  mean_squared_error(y_test, val_pred_svr),
                  r2_score(y_train, train_pred_svr),
                  r2_score(y_test, val_pred_svr),
                  svr.get_params()]

i += 1

In [ ]:
# Hyperparameter tuning

svr=SVR()

grid_params_svr =  {'C': [1, 10], 
                    'kernel': [ "rbf", "linear"],
                   "gamma": [0.0001,0.001, 0.01, 0.1],
                   "epsilon": [0.01, 0.05, 0.1, 0.5]}
scoring = {'R2': 'r2', 'MSE': "neg_mean_squared_error"}

CV_svr = GridSearchCV(estimator=svr, param_grid=grid_params_svr, cv = 3, n_jobs = -1, verbose = 0, scoring= scoring, refit = "R2",return_train_score=True)
CV_svr.fit(X_train, y_train)

print(CV_svr.best_params_)

In [ ]:
svr = CV_svr.best_estimator_
svr.fit(X_train, y_train)

# Predict
val_pred_svr = svr.predict(X_test)

print("Validation MSE:", round(mean_squared_error(y_test, val_pred_svr),4))
print("Validation r2:", round(r2_score(y_test, val_pred_svr),4))

In [ ]:
results.loc[i] = ["SVR2",
                  mean_squared_error(y_train, svr.predict(X_train)),
                  mean_squared_error(y_test, val_pred_svr),
                  r2_score(y_train, svr.predict(X_train)),
                  r2_score(y_test, val_pred_svr),
                  CV_svr.get_params()]

i += 1

# Multilayer Perceptron

In [ ]:
# "grid search"
#lr = [ 10 ** x for x in range(-5,1)]
#settings = []
#for act in ["relu", "tanh"]:
#    for eta0 in lr:
#        for bs in [64, 128, 256]:
#            for e in [100,150,200]:
#                model = models.Sequential()
#                model.add(layers.Dense(128, input_shape=(X_train.shape[1],), activation = act))
#                model.add(layers.Dense(256, activation = act))
#                model.add(layers.Dense(256, activation = act))
#                model.add(layers.Dense(256, activation = act))
#                model.add(layers.Dense(1, activation = "linear"))
#                opt =  optimizers.Adam(learning_rate=eta0)
#                model.compile(optimizer = opt, loss = "mean_squared_error", metrics = ["mean_squared_error"])
#                model.fit(X_train, y_train, batch_size=bs, epochs=e, verbose=1)
#                y_pred = model.predict(X_test, verbose=0)
#                r2 = r2_score(y_test, y_pred)
#                mse = mean_squared_error(y_test,y_pred)
#                settings.append((dns, act, eta0, bs, e, mse,r2))
#                print(settings[-1])

#best_r2 =  max(settings, key=lambda x: x[-1])
#print("Best settings according to R2 {}".format(best_r2))

In [ ]:
# Separating X and y
X = airbnb_walk.drop('price', axis=1)
y = airbnb_walk.price

# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=123)

# Scale 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
nn1_relu = models.Sequential()
nn1_relu.add(layers.Dense(64, input_shape=(X_train.shape[1],), activation='relu'))
nn1_relu.add(layers.Dense(64, activation='relu'))
nn1_relu.add(layers.Dense(32, activation='relu'))
nn1_relu.add(layers.Dense(1, activation='linear'))

# opt =  optimizers.Adam(learning_rate=0.001)
nn1_relu.compile(optimizer = "sgd", loss = "mean_squared_error", metrics = ["mean_squared_error"])

# fit the model
nn1_relu1_history = nn1_relu.fit(X_train,
                      y_train,
                      epochs=300,
                      batch_size=64,
                      validation_split = 0.2,
                      verbose = 0)

In [ ]:
history1 = pd.DataFrame(nn1_relu1_history)
plt.figure(figsize=(6,6));
loss_plt = plt.plot(history1["val_loss"], label = "Loss");
accuracy_plt = plt.plot(history1["mean_squared_error"], label = "MSE");
plt.legend(bbox_to_anchor=(1, 1))
plt.title("Loss vs MSE");
plt.show()

In [ ]:
# Building the model
nn1_relu2 = models.Sequential()
nn1_relu2.add(layers.Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
nn1_relu2.add(layers.Dense(256, activation='relu'))
nn1_relu2.add(layers.Dense(256, activation='relu'))
nn1_relu2.add(layers.Dense(256, activation='relu'))
nn1_relu2.add(layers.Dense(1, activation='linear'))

# Compiling the model
nn1_relu2.compile(loss='mean_squared_error',
            optimizer='sgd',
            metrics=['mean_squared_error'])

# Training the model
nn1_relu2_history = nn1_relu2.fit(X_train,
                  y_train,
                  epochs=300,
                  batch_size=128,
                  validation_split = 0.2)

In [ ]:
history2 = pd.DataFrame(nn1_relu2_history)
plt.figure(figsize=(6,6));
loss_plt = plt.plot(history2["val_loss"], label = "Loss");
accuracy_plt = plt.plot(history2["mean_squared_error"], label = "MSE");
plt.legend(bbox_to_anchor=(1, 1))
plt.title("Loss vs MSE");
plt.show()

In [ ]:
# Building the model
nn1_relu3 = models.Sequential()
nn1_relu3.add(layers.Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
nn1_relu3.add(layers.Dense(256, activation='relu'))
nn1_relu3.add(layers.Dense(256, activation='relu'))
nn1_relu3.add(layers.Dense(256, activation='relu'))
nn1_relu3.add(layers.Dense(1, activation='linear'))

# Compiling the model
nn1_relu3.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])

# Training the model
nn1_relu3_history = nn1_relu3.fit(X_train,
                  y_train,
                  epochs=300,
                  batch_size=128,
                  validation_split = 0.2)

In [ ]:
# Building the model
nn1_relu4 = models.Sequential()
nn1_relu4.add(layers.Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
nn1_relu4.add(layers.Dense(256, activation='relu'))
nn1_relu4.add(layers.Dense(256, activation='relu'))
nn1_relu4.add(layers.Dense(512, activation='relu'))
nn1_relu4.add(layers.Dense(1, activation='linear'))

# Compiling the model
nn1_relu4.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error']) 

# Training the model
nn1_relu4_history = nn1_relu4.fit(X_train,
                  y_train,
                  epochs=200,
                  batch_size=128,
                  validation_split = 0.2)

#### L1 regularization: 0.0001

In [ ]:
nn1_relu4_r = models.Sequential()
nn1_relu4_r.add(layers.Dense(128, input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l1(0.0001), activation='relu'))
nn1_relu4_r.add(layers.Dense(256,kernel_regularizer=regularizers.l1(0.0001), activation='relu'))
nn1_relu4_r.add(layers.Dense(256,kernel_regularizer=regularizers.l1(0.0001), activation='relu'))
nn1_relu4_r.add(layers.Dense(512,kernel_regularizer=regularizers.l1(0.0001), activation='relu'))
nn1_relu4_r.add(layers.Dense(1,kernel_regularizer=regularizers.l1(0.0001), activation='linear'))

# Compiling the model
nn1_relu4_r.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])

# Training the model
nn1_relu4_r_history = nn1_relu4_r.fit(X_train,
                  y_train,
                  epochs=200,
                  batch_size=128,
                  validation_split = 0.2)

#### L1 regularization: 0.0005

In [ ]:
nn1_relu4_r2 = models.Sequential()
nn1_relu4_r2.add(layers.Dense(128, input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l1(0.0005), activation='relu'))
nn1_relu4_r2.add(layers.Dense(256,kernel_regularizer=regularizers.l1(0.0005), activation='relu'))
nn1_relu4_r2.add(layers.Dense(256,kernel_regularizer=regularizers.l1(0.0005), activation='relu'))
nn1_relu4_r2.add(layers.Dense(512,kernel_regularizer=regularizers.l1(0.0005), activation='relu'))
nn1_relu4_r2.add(layers.Dense(1,kernel_regularizer=regularizers.l1(0.0005), activation='linear'))

# Compiling the model
nn1_relu4_r2.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])

# Training the model
nn1_relu4_r2_history = nn1_relu4_r2.fit(X_train,
                  y_train,
                  epochs=200,
                  batch_size=128,
                  validation_split = 0.2)

#### L1 reg. 0.001

In [ ]:
nn1_relu4_r3 = models.Sequential()
nn1_relu4_r3.add(layers.Dense(128, input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l1(0.001), activation='relu'))
nn1_relu4_r3.add(layers.Dense(256,kernel_regularizer=regularizers.l1(0.001), activation='relu'))
nn1_relu4_r3.add(layers.Dense(256,kernel_regularizer=regularizers.l1(0.001), activation='relu'))
nn1_relu4_r3.add(layers.Dense(512,kernel_regularizer=regularizers.l1(0.001), activation='relu'))
nn1_relu4_r3.add(layers.Dense(1,kernel_regularizer=regularizers.l1(0.001), activation='linear'))

# Compiling the model
nn1_relu4_r3.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])
# Printing the model summary
print(nn1_relu4_r3.summary())

# Visualising the neural network
#SVG(model_to_dot(nn2, show_layer_names=False, show_shapes=True).create(prog='dot', format='svg'))

# Training the model
nn1_relu4_r3_history = nn1_relu4_r3.fit(X_train,
                  y_train,
                  epochs=200,
                  batch_size=128,
                  validation_split = 0.2)

#### L1 reg. 0.005

In [ ]:
nn1_relu4_r4 = models.Sequential()
nn1_relu4_r4.add(layers.Dense(128, input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l1(0.005), activation='relu'))
nn1_relu4_r4.add(layers.Dense(256,kernel_regularizer=regularizers.l1(0.005), activation='relu'))
nn1_relu4_r4.add(layers.Dense(256,kernel_regularizer=regularizers.l1(0.005), activation='relu'))
nn1_relu4_r4.add(layers.Dense(512,kernel_regularizer=regularizers.l1(0.005), activation='relu'))
nn1_relu4_r4.add(layers.Dense(1,kernel_regularizer=regularizers.l1(0.005), activation='linear'))

# Compiling the model
nn1_relu4_r4.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])
# Printing the model summary
print(nn1_relu4_r4.summary())

# Visualising the neural network
#SVG(model_to_dot(nn2, show_layer_names=False, show_shapes=True).create(prog='dot', format='svg'))

# Training the model
nn1_relu4_r4_history = nn1_relu4_r4.fit(X_train,
                  y_train,
                  epochs=200,
                  batch_size=128,
                  validation_split = 0.2)

#### L1 reg. 0.01

In [ ]:
nn1_relu4_r5 = models.Sequential()
nn1_relu4_r5.add(layers.Dense(128, input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l1(0.01), activation='relu'))
nn1_relu4_r5.add(layers.Dense(256,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
nn1_relu4_r5.add(layers.Dense(256,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
nn1_relu4_r5.add(layers.Dense(512,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
nn1_relu4_r5.add(layers.Dense(1,kernel_regularizer=regularizers.l1(0.01), activation='linear'))

# Compiling the model
nn1_relu4_r5.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])
# Printing the model summary
print(nn1_relu4_r5.summary())

# Visualising the neural network
#SVG(model_to_dot(nn2, show_layer_names=False, show_shapes=True).create(prog='dot', format='svg'))

# Training the model
nn1_relu4_r5_history = nn1_relu4_r5.fit(X_train,
                  y_train,
                  epochs=200,
                  batch_size=128,
                  validation_split = 0.2)

#### L2 regularization:  0.0001

In [ ]:
nn1_relu4_r_l2_1 = models.Sequential()
nn1_relu4_r_l2_1.add(layers.Dense(128, input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
nn1_relu4_r_l2_1.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
nn1_relu4_r_l2_1.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
nn1_relu4_r_l2_1.add(layers.Dense(512,kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
nn1_relu4_r_l2_1.add(layers.Dense(1,kernel_regularizer=regularizers.l2(0.0001), activation='linear'))

# Compiling the model
nn1_relu4_r_l2_1.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])

# Training the model
nn1_relu4_r_l2_1_history = nn1_relu4_r_l2_1.fit(X_train,
                  y_train,
                  epochs=150,
                  batch_size=256,
                  validation_split = 0.1)

#### L2 regularization:  0.001

In [ ]:
nn1_relu4_r_l2_2 = models.Sequential()
nn1_relu4_r_l2_2.add(layers.Dense(128, input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l2(0.001), activation='relu'))
nn1_relu4_r_l2_2.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.001), activation='relu'))
nn1_relu4_r_l2_2.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.001), activation='relu'))
nn1_relu4_r_l2_2.add(layers.Dense(512,kernel_regularizer=regularizers.l2(0.001), activation='relu'))
nn1_relu4_r_l2_2.add(layers.Dense(1,kernel_regularizer=regularizers.l2(0.001), activation='linear'))

# Compiling the model
nn1_relu4_r_l2_2.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])
# Printing the model summary
print(nn1_relu4_r_l2_2.summary())

# Visualising the neural network
#SVG(model_to_dot(nn2, show_layer_names=False, show_shapes=True).create(prog='dot', format='svg'))

# Training the model
nn1_relu4_r_l2_2_history = nn1_relu4_r_l2_2.fit(X_train,
                  y_train,
                  epochs=150,
                  batch_size=256,
                  validation_split = 0.1)

#### L2 regularization:  0.01

In [ ]:
nn1_relu4_r_l2_3 = models.Sequential()
nn1_relu4_r_l2_3.add(layers.Dense(128, input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l2(0.01), activation='relu'))
nn1_relu4_r_l2_3.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
nn1_relu4_r_l2_3.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
nn1_relu4_r_l2_3.add(layers.Dense(512,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
nn1_relu4_r_l2_3.add(layers.Dense(1,kernel_regularizer=regularizers.l2(0.01), activation='linear'))

# Compiling the model
nn1_relu4_r_l2_3.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ["mean_squared_error"])

# Training the model
nn1_relu4_r_l2_3_history = nn1_relu4_r_l2_3.fit(X_train,
                  y_train,
                  epochs=150,
                  batch_size=256,
                  validation_split = 0.2)

In [ ]:
#### Tanh activation

In [ ]:
# Best setting with tanh activation:

In [ ]:
nn1_tanh_r = models.Sequential()
nn1_tanh_r.add(layers.Dense(128, input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l2(0.007), activation='tanh'))
nn1_tanh_r.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.007), activation='tanh'))
nn1_tanh_r.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.007), activation='tanh'))
nn1_tanh_r.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.007), activation='tanh'))
nn1_tanh_r.add(layers.Dense(512,kernel_regularizer=regularizers.l2(0.007), activation='tanh'))
nn1_tanh_r.add(layers.Dense(1,kernel_regularizer=regularizers.l2(0.007), activation='linear'))

# Compiling the model
opt =  optimizers.Adam(learning_rate=0.01)
nn1_tanh_r.compile(optimizer = opt, loss = "mean_squared_error", metrics = ["mean_squared_error"])
# Printing the model summary

# Training the model
nn1_tanh_r_history = nn1_tanh_r.fit(X_train,
                  y_train,
                  epochs=150,
                  batch_size=256,
                  validation_split = 0.2)